TensorFlow is an open source, distributed numerical computation framework released by Google that is mainly intended to alleviate the painful details of implementing a neural network.

Below some exercises taken from Chapter 2 of the book **Natural Language Processing
with TensorFlow** *Second Edition*

In [2]:
import tensorflow as tf
import numpy as np
import os
print(tf.__version__)

2.12.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()

Mounted at /content/drive


'/content'

In [ ]:
#Implementing the sigmoid function
@tf.function
def layer(x, W, b):
  # Building the graph
  h = tf.nn.sigmoid(tf.matmul(x,W) + b) # Operation to perform
  return h

A Python decorator provides a clean way
to call another function whenever you call the decorated function. In other words, every time the layer() function is called, tf.function() is called.

When the layer() function is passing through tf.function(), TensorFlow will trace the content (in other words, the operations and data) in the function and build a computational graph
automatically.

tf.function() is a multi-stage process, where it first builds the dataflow graph and then executes it. Additionally, since TensorFlow traces each line in the function, if something
goes wrong, TensorFlow can point to the exact line that is causing the issue.

In [ ]:
x = np.array([[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]], dtype=np.float32)
# Variable
init_w = tf.initializers.RandomUniform(minval=-0.1, maxval=0.1)(shape=[10,5])
W = tf.Variable(init_w, dtype=tf.float32, name='W')
# Variable
init_b = tf.initializers.RandomUniform()(shape=[5])
b = tf.Variable(init_b, dtype=tf.float32, name='b')
h = layer(x,W,b)
print(f"h = {h.numpy()}")

print(f"h = {h}")
print(f"h is of type {type(h)}")

h = [[0.49629167 0.50309455 0.4909432  0.47080386 0.50476456]]
h = [[0.49629167 0.50309455 0.4909432  0.47080386 0.50476456]]
h is of type <class 'tensorflow.python.framework.ops.EagerTensor'>


TensorFlow executes its operations “eagerly”, or immediately after the
layer() function is called. This is a special mode in TensorFlow known as eager execution mode.
This was an optional mode for TensorFlow 1, but has been made the default in TensorFlow 2

**Defining inputs in TensorFlow**

There are three different ways you can feed data to a TensorFlow program:

• Feeding data as NumPy arrays

    x = np.array([[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]], dtype=np.float32)

• Feeding data as TensorFlow tensors

    x = tf.constant(value=[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]],dtype=tf.float32,name='x')

• Using the tf.data API to create an input pipeline


**tf.data** API provides convenient functions that can be used to easily load and transform the data, creating pipelines
designed for heavy-duty programs that need to process a lot of data, including large datasets that do not fit in memory. Furthermore, it streamlines your data ingestion code with the model training.

In [ ]:
filenames = [f"./iris.data.{i}" for i in range(1,4)]
dataset = tf.data.experimental.CsvDataset(filenames, [tf.float32,tf.float32, tf.float32, tf.float32, tf.string])
#The dataset reader takes in a list of filenames and another list that specifies the data types of each column in the dataset.

dataset = dataset.map(lambda x1,x2,x3,x4,y: (tf.stack([x1,x2,x3,x4]), y))
#We are using lambda functions to separate out x1,x2,x3,x4 into one dataset and y to another
#dataset, along with the dataset.map() function.

In [ ]:
dataset = dataset.filter(lambda x,y: tf.reduce_min(x)>0)
#for next_element in dataset: in order to loop the mapped elements in dataset

**Comparison operations**

In [ ]:
# Let's assume the following values for x and y
# x (2-D tensor) => [[1,2],[3,4]]
# y (2-D tensor) => [[4,3],[3,2]]
x = tf.constant([[1,2],[3,4]], dtype=tf.int32)
y = tf.constant([[4,3],[3,2]], dtype=tf.int32)
print(x)
print(y)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[4 3]
 [3 2]], shape=(2, 2), dtype=int32)


In [ ]:
# Checks if two tensors are equal element-wise and returns a boolean
# tensor
# x_equal_y => [[False,False],[True,False]]
x_equal_y = tf.equal(x, y, name=None)
print(x_equal_y)

tf.Tensor(
[[False False]
 [ True False]], shape=(2, 2), dtype=bool)


In [ ]:
# Checks if x is less than y element-wise and returns a boolean tensor
# x_less_y => [[True,True],[False,False]]
x_less_y = tf.less(x, y, name=None)
print(x_less_y)

tf.Tensor(
[[ True  True]
 [False False]], shape=(2, 2), dtype=bool)


In [ ]:
# Checks if x is greater or equal than y element-wise and returns a
# boolean tensor
# x_great_equal_y => [[False,False],[True,True]]
x_great_equal_y = tf.greater_equal(x, y, name=None)
print(x_great_equal_y)

tf.Tensor(
[[False False]
 [ True  True]], shape=(2, 2), dtype=bool)


In [ ]:
# Selects elements from x and y depending on whether,
# the condition is satisfied (select elements from x)
# or the condition failed (select elements from y)
condition = tf.constant([[True,False],[True,False]],dtype=tf.bool)
# x_cond_y => [[1,3],[3,2]]
x_cond_y = tf.where(condition, x, y, name=None)
print(x_cond_y)

tf.Tensor(
[[1 3]
 [3 2]], shape=(2, 2), dtype=int32)


**Keras: The model building API of TensorFlow**

Keras was developed as a separate library that provides high-level building blocks to build models conveniently.

Keras’s primary focus is model building. For that, Keras provides several different APIs with varying degrees of flexibility and complexity. Choosing the right API for the job will require sound
knowledge of the limitations of each API as well as experience.

The APIs provided by Keras are:

*   **Sequential API** – The most easy-to-use API. You simply stack layers on top of each other to create a model.
*   **Functional API** – Provides more flexibility by allowing you to define  custom models that can have multiple input layers/multiple output layers.
*   **Sub-classing API** – Allows you to define custom reusable layers/models as Python classes. This is the most flexible API, but it requires strong familiarity with the API and raw TensorFlow operations to use it correctly.

**Sequential API**

In [ ]:
%%time
# Simply define your model as a list of layers.
# The first element in the list is the closest to the input, whereas the last is the output layer.
model = tf.keras.Sequential([
            tf.keras.layers.Dense(500, activation='relu', input_dim=784),
            tf.keras.layers.Dense(250, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
                            ])

#In the code above, we have three layers. The first layer has 500 output nodes and takes in a
#vector of 784 elements as the input.
#The second layer is automatically connected to the first one,
#whereas the last layer is connected to the second layer.
#All of these layers are fully-connected layers, where all input nodes are connected to all output nodes.
type(model)

CPU times: user 58.9 ms, sys: 186 µs, total: 59.1 ms
Wall time: 103 ms


keras.engine.sequential.Sequential

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 500)               392500    
                                                                 
 dense_4 (Dense)             (None, 250)               125250    
                                                                 
 dense_5 (Dense)             (None, 10)                2510      
                                                                 
Total params: 520,260
Trainable params: 520,260
Non-trainable params: 0
_________________________________________________________________


#Implementing a neural network using Keras and MNIST

In [5]:
path = os.getcwd()+'/drive/MyDrive/colab_files/'
os.makedirs(path+'mnist_data', exist_ok=True)

In [6]:
%%time
#Downloading the data and storing it as numpy.ndarray objects.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(
                                          path=os.path.join(path, 'mnist_data', 'mnist.npz')
                                                                        )

11490434/11490434 [==============================] - 0s 0us/step
CPU times: user 456 ms, sys: 83.2 ms, total: 539 ms
Wall time: 729 ms


In [7]:
print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (60000, 28, 28)
y_train shape:  (60000,)
x_test shape:  (10000, 28, 28)
y_test shape:  (10000,)


In [8]:
# Reshaping x_train and x_test tensors so that each image is represented
# as a 1D vector
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)

x_train shape:  (60000, 784)
x_test shape:  (10000, 784)


In [9]:
# Standardizing x_train and x_test tensors
x_train = (x_train - np.mean(x_train, axis=1, keepdims=True))/np.std(x_train, axis=1, keepdims=True)
x_test = (x_test - np.mean(x_test, axis=1, keepdims=True))/np.std(x_test, axis=1, keepdims=True)

In [11]:
# One hot encoding y_train and y_test
num_labels = 10
y_onehot_train = np.zeros((y_train.shape[0], num_labels),dtype=np.float32)
y_onehot_train[np.arange(y_train.shape[0]), y_train] = 1.0
y_onehot_test = np.zeros((y_test.shape[0], num_labels), dtype=np.float32)
y_onehot_test[np.arange(y_test.shape[0]), y_test] = 1.0

In [12]:
#The network is a fully connected neural
#network with 3 layers having 500, 250, and 10 nodes, respectively. The first two layers will use
#ReLU activation, whereas the last layer uses softmax. To implement this, we are going to use the
#the Sequential API.

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(500, activation='relu'),
                            tf.keras.layers.Dense(250, activation='relu'),
                            tf.keras.layers.Dense(10, activation='softmax')
                            ])

#For a fully connected
#network, we only need Dense layers that mimic the computations of a hidden layer in a fully
#connected network. With the model defined, we need to compile this model with an appropriate
#loss function, an optimizer, and, optionally, performance metrics:

optimizer = tf.keras.optimizers.RMSprop()
loss_fn = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['acc'])

In [13]:
%%time
#Training the model

batch_size = 100
num_epochs = 10

train_history = model.fit(
                          x=x_train,
                          y=y_onehot_train,
                          batch_size=batch_size,
                          epochs= num_epochs,
                          validation_split=0.2
                          )

Epoch 1/10
480/480 [==============================] - 8s 13ms/step - loss: 0.2289 - acc: 0.9293 - val_loss: 0.1402 - val_acc: 0.9576
Epoch 2/10
480/480 [==============================] - 7s 14ms/step - loss: 0.0872 - acc: 0.9731 - val_loss: 0.1016 - val_acc: 0.9710
Epoch 3/10
480/480 [==============================] - 5s 11ms/step - loss: 0.0579 - acc: 0.9820 - val_loss: 0.0883 - val_acc: 0.9750
Epoch 4/10
480/480 [==============================] - 7s 15ms/step - loss: 0.0413 - acc: 0.9867 - val_loss: 0.1153 - val_acc: 0.9722
Epoch 5/10
480/480 [==============================] - 6s 12ms/step - loss: 0.0306 - acc: 0.9899 - val_loss: 0.1217 - val_acc: 0.9741
Epoch 6/10
480/480 [==============================] - 7s 14ms/step - loss: 0.0236 - acc: 0.9925 - val_loss: 0.1260 - val_acc: 0.9735
Epoch 7/10
480/480 [==============================] - 6s 12ms/step - loss: 0.0181 - acc: 0.9942 - val_loss: 0.1170 - val_acc: 0.9792
Epoch 8/10
480/480 [==============================] - 5s 11ms/step - 

In [14]:
%%time
#testing the model
test_res = model.evaluate(
                          x=x_test,
                          y=y_onehot_test,
                          batch_size=batch_size
                          )

100/100 [==============================] - 1s 9ms/step - loss: 0.1242 - acc: 0.9794
CPU times: user 1.04 s, sys: 51.9 ms, total: 1.09 s
Wall time: 1.98 s
